In [1]:
import pandas as pd
import os

df = pd.read_csv("augmented_common_containers_with_types.csv")

output_dir = "mannheim_container_ts"
os.makedirs(output_dir, exist_ok=True)

for label, group in df.groupby("Label"):
    safe_label = label.replace(" ", "_").replace("#", "")
    filename = f"{output_dir}/{safe_label}.csv"
    group.to_csv(filename, index=False)



In [2]:
def get_latest_füllstand(df, label, current_datetime):
    filtered_df = df[df["Label"] == label].copy()

    filtered_df["datetime"] = pd.to_datetime(
        filtered_df["Datum"].astype(str) + " " + filtered_df["Uhrzeit"].astype(str),
        format="%Y-%m-%d %H:%M"
    )

    past_rows = filtered_df[filtered_df["datetime"] < current_datetime]

    if not past_rows.empty:
        latest_row = past_rows.sort_values("datetime").iloc[-1]
        return latest_row["Füllstand (%)"]
    else:
        return None


In [4]:
import pandas as pd
import folium
import os
from datetime import datetime

current_timestamp = datetime(2024, 1, 15, 16, 30)
location_csv = "altglascontainer-mannheim-clean.csv"
ts_folder = "mannheim_container_ts"

location_df = pd.read_csv(location_csv)

center_lat = location_df["Latitude"].mean()
center_lon = location_df["Longitude"].mean()
map_mannheim = folium.Map(location=[center_lat, center_lon], zoom_start=13)

def get_latest_fill_level(file_path, current_time):
    df = pd.read_csv(file_path)
    df["datetime"] = pd.to_datetime(df["Datum"] + " " + df["Uhrzeit"], format="%Y-%m-%d %H:%M")
    past_df = df[df["datetime"] <= current_time]
    if not past_df.empty:
        return past_df.sort_values("datetime").iloc[-1]["Füllstand (%)"]
    return None

def sanitize_label(label):
    return label.replace(" ", "_").replace("#", "")

for _, row in location_df.iterrows():
    label = row["Label"]
    safe_label = sanitize_label(label)
    ts_file = os.path.join(ts_folder, f"{safe_label}.csv")

    lat = row["Latitude"]
    lon = row["Longitude"]

    if not os.path.exists(ts_file):
        print(f"Missing timeseries file for: {label} (expected: {ts_file})")
        continue

    fill_level = get_latest_fill_level(ts_file, current_timestamp)
    print(f"Label: {label}, Fill Level: {fill_level} at {current_timestamp}")

    if fill_level is None:
        continue

    if fill_level >= 80:
        color = "red"
    elif fill_level >= 40:
        color = "orange"
    else:
        color = "green"

    popup_text = f"{label}<br>Fill: {fill_level:.1f}%"
    folium.CircleMarker(
        location=[lat, lon],
        radius=8,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=popup_text
    ).add_to(map_mannheim)


Missing timeseries file for: Depot #1 (expected: mannheim_container_ts/Depot_1.csv)
Label: Altglas Container #1, Fill Level: 33.7 at 2024-01-15 16:30:00
Label: Altglas Container #2, Fill Level: 24.1 at 2024-01-15 16:30:00
Label: Altglas Container #3, Fill Level: 33.5 at 2024-01-15 16:30:00
Label: Altglas Container #4, Fill Level: 17.7 at 2024-01-15 16:30:00
Label: Altglas Container #5, Fill Level: 39.0 at 2024-01-15 16:30:00
Label: Altglas Container #6, Fill Level: 28.0 at 2024-01-15 16:30:00
Label: Altglas Container #7, Fill Level: 66.9 at 2024-01-15 16:30:00
Label: Altglas Container #8, Fill Level: 52.1 at 2024-01-15 16:30:00
Label: Altglas Container #9, Fill Level: 29.2 at 2024-01-15 16:30:00
Label: Altglas Container #10, Fill Level: 34.7 at 2024-01-15 16:30:00
Label: Altglas Container #11, Fill Level: 44.4 at 2024-01-15 16:30:00
Label: Altglas Container #12, Fill Level: 26.8 at 2024-01-15 16:30:00
Label: Altglas Container #13, Fill Level: 44.3 at 2024-01-15 16:30:00
Label: Altglas 

In [5]:
map_mannheim